### 1 - Import test data

In [16]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2023-9-7", end="2023-11-1", interval='15m')
dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-09-07 00:00:00,1.072731,1.072846,1.072616,1.072616,1.072616,0
2023-09-07 00:15:00,1.072616,1.072616,1.072501,1.072616,1.072616,0
2023-09-07 00:30:00,1.072616,1.072731,1.072501,1.072501,1.072501,0
2023-09-07 00:45:00,1.072501,1.072616,1.072501,1.072616,1.072616,0
2023-09-07 01:00:00,1.072731,1.072731,1.072501,1.072501,1.072501,0
...,...,...,...,...,...,...
2023-10-31 22:45:00,1.058089,1.058313,1.058089,1.058201,1.058201,0
2023-10-31 23:00:00,1.058313,1.058313,1.058089,1.058089,1.058089,0
2023-10-31 23:15:00,1.058089,1.058201,1.058089,1.058089,1.058089,0


### 2 - Define your signal function

In [17]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]

    # Bearish Pattern
    if (open>close and
    previous_open<previous_close and
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and
        previous_open>previous_close and
        close>previous_open and
        open<=previous_close):
        return 2

    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

In [18]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

signal
0    3399
1     154
2     143
Name: count, dtype: int64

### 3 - Connect to the market and execute trades

In [19]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [20]:
access_token= "cd7e21b015cb734770f5fcfea9da9c9f-49a95de76ed5cab29ee0d33cee5f004d"
accountID = "101-001-27332384-001" #your account ID here
def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


True
True
True


In [24]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])

    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#

    # EXECUTING ORDERS
    client = API(access_token)

    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])

    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio

    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [25]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()

      Open    Close     High      Low
0  1.07267  1.07297  1.07297  1.07259
1  1.07297  1.07304  1.07320  1.07294
1.0735499999999998    1.07277    1.07251    1.0732899999999999
{'orderCreateTransaction': {'id': '14', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '14', 'requestID': '25141190429682750', 'time': '2023-11-05T04:46:43.548031521Z', 'type': 'MARKET_ORDER', 'instrument': 'EUR_USD', 'units': '1000', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '1.07355', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '1.07277', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderCancelTransaction': {'id': '15', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '14', 'requestID': '25141190429682750', 'time': '2023-11-05T04:46:43.548031521Z', 'type': 'ORDER_CANCEL', 'orderID': '14', 'reason': 'MARKET_HALTED'}, 'relatedTransactionIDs': ['14', '15'], 'lastTransactionID': '15'}
